In [3]:
import networkx as nx
from typing import Iterable, Literal
import pandas as pd


def read_time_stamped_csv(
    uri: str,
    format: Literal["datetime", "numeric"] = "numeric",
    columns=("Source", "Target", "Timestamp"),
) -> tuple[nx.MultiDiGraph, Iterable[tuple[str, str, dict]]]:
    source_column, target_column, timestamp_column = columns

    dataset_df = pd.read_csv(uri)
    if format == "datetime":
        dataset_df[timestamp_column] = pd.to_datetime(
            dataset_df[timestamp_column], format="%m/%d/%y %I:%M %p"
        )

    G = nx.from_pandas_edgelist(
        dataset_df,
        source=source_column,
        target=target_column,
        edge_attr=[timestamp_column],
        create_using=nx.MultiDiGraph,
    )

    return (G, sorted(G.edges(data=True), key=lambda x: x[2][timestamp_column]))

In [4]:
G_college_msg, college_msg_edges = read_time_stamped_csv("college-msg.csv", "datetime")

In [5]:
def PaCo(
    sorted_edges: Iterable[tuple[str, str, dict]],
    delta_time: int,
    max_path_length: int,
    verbose=False,
):
    counts = dict()
    window = list()
    for source, target, time in sorted_edges:
        time = time["Timestamp"]
        current_counts = dict()
        current_counts[(source, target)] = 1

        # remove items from the window that are too old (from looking at delta_time)
        window = [
            (source_window, target_window, time_window, dict_window)
            for source_window, target_window, time_window, dict_window in window
            if time_window >= time - delta_time
        ]
        for source_window, target_window, time_window, counts_window in window:
            # skip if edge in window doesn't connect or is not formed after time
            if target_window != source or time <= time_window:
                continue

            # look through the window which paths can be exteded with current edge
            for path in counts_window.keys():
                # Check if the length of path is not too large to add a node to it given max_path_length
                if len(path) - 1 >= max_path_length:
                    continue
                # combine path in window with current edge, and count the path variations
                combined_path = (*path, target)
                current_counts[combined_path] = (
                    current_counts.get(combined_path, 0) + counts_window[path]
                )

        if verbose:
            print("current counts", current_counts)

        # Add current_counts to counts
        for key in current_counts:
            counts[key] = counts.get(key, 0) + current_counts[key]

        # Increment window
        window.append((source, target, time, current_counts))
    return counts

In [6]:
# Validate algorithm against the expected causal path counts from the PaCo paper
def test_causal_path_algorithm(algorithm):
    _, test_edges = read_time_stamped_csv("test-edges.csv", "numeric")
    result = algorithm(test_edges, 2, 3)
    print(result)
    expected_result = {
        ("a", "b"): 2,
        ("b", "c"): 2,
        ("c", "d"): 1,
        ("d", "c"): 2,
        ("c", "b"): 1,
        ("b", "a"): 1,
        ("a", "b", "a"): 2,
        ("a", "b", "c"): 2,
        ("b", "c", "d"): 1,
        ("c", "b", "c"): 1,
        ("d", "c", "b"): 1,
        ("d", "c", "d"): 2,
    }
    try:
        assert result == expected_result
        print("Tests succeeded ✅")
    except:
        print("Tests failed ❌")

In [7]:
test_causal_path_algorithm(PaCo)

{('a', 'b'): 2, ('b', 'a'): 1, ('a', 'b', 'a'): 2, ('b', 'c'): 2, ('a', 'b', 'c'): 2, ('d', 'c'): 2, ('c', 'd'): 1, ('b', 'c', 'd'): 1, ('a', 'b', 'c', 'd'): 2, ('d', 'c', 'd'): 2, ('c', 'b'): 1, ('d', 'c', 'b'): 1, ('c', 'b', 'c'): 1, ('d', 'c', 'b', 'c'): 1}
Tests failed ❌


In [ ]:
from pathpy import TemporalNetwork, Paths
from collections import Counter


def pathpy_algorithm(
    sorted_edges: Iterable[tuple[str, str, dict]],
    delta_time: int,
    max_path_length: int,
    verbose=False,
):
    temporal_network = TemporalNetwork()
    for source, target, time in sorted_edges:
        time = time["Timestamp"]
        temporal_network.addEdge(source, target, int(time))

    # for e in temporal_network.tedges:
    #     print(e)
    # temporal_network_dag, _ = DAG.from_temporal_network(
    #     temporal_network, delta=delta_time
    # )

    # print(temporal_network_dag)

    causal_paths = Paths.fromTemporalNetwork(
        temporal_network, delta_time, max_path_length
    )
    print(max_path_length)
    for p, counts in causal_paths.paths[max_path_length - 1].items():
        print(f"{p} -> {counts[1]}")

    print(causal_paths.getSequence(","))

    for l in causal_paths.paths:
        for p in causal_paths.paths[l]:
            if causal_paths.paths[l][p][1] > 0:
                print("{0} -> {1}".format(p, causal_paths.paths[l][p][1]))


test_causal_path_algorithm(pathpy_algorithm)

2024-11-29 11:25:26 [2]	Extracting time-respecting paths up to length 3 for delta = 2 ...
2024-11-29 11:25:26 [2]	Calculating sub path statistics ... 
2024-11-29 11:25:26 [2]	finished.
3
('a', 'b', 'a') -> 2
('d', 'c', 'd') -> 2
('c', 'b', 'c') -> 1
('b', 'c', 'd') -> 1
2024-11-29 11:25:26 [2]	Concatenating paths to sequence ...
2024-11-29 11:25:26 [2]	finished
['a', 'b', 'a', ',', 'a', 'b', 'a', ',', 'd', 'c', 'd', ',', 'd', 'c', 'd', ',', 'c', 'b', 'c', ',', 'b', 'c', 'd', ',']
None
Tests failed ❌
